In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!pip install lightgbm pandas-profiling==2.7.1

In [16]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import pandas_profiling as prof
from datetime import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error
import json
from dateutil.relativedelta import relativedelta

%matplotlib inline
pd.set_option('max_columns', 500)


PATH = './drive/MyDrive/Colab Notebooks/Lina/'
SPLIT_TIPE = 'OOS'
DESCRIBE = 'Baseline_housebase_'

In [17]:
def ti():
    print(dt.now().replace(microsecond=0))  
def get_zaic(df,*args):
    ti()
    if not(isinstance(df,pd.DataFrame) or isinstance(df,pd.Series)):
        print('Не могу вернуть размер для объекта с типом',type(df))
    elif isinstance(df,pd.Series) or isinstance(df,pd.DataFrame) and df.shape[1] == 1:
        print(round(df.memory_usage(deep=True)/1024**2,2),'Mb')
    else:
        print(round(df.memory_usage(deep=True).sum()/1024**2,2),'Mb')
    print(df.shape)


## Загрузим и посмотрим на данные 

Описание полей в файле price_train.tsv:

* **city_quadkey** - полигон, его координаты можно его узнать из скрипта quadkey.py
* **month** - месяц, за который указаны данные в датасете
* **apartment_ratio** - доля квартир в этом районе в этом месяце со статусом "апартаменты" 
* **offers_in_sites_ratio** - доля офферов в ЖК в этом районе в этом месяце
* **unique_active_buildings_count** - количество домов, в которых есть квартиры в продаже в этом районе в этом месяце
* **AVG_PRICE_SQM** - средняя цена за квадратный метр	в этом районе в этом месяце
* **unique_active_building_series_count** - количество уникальных серий домов, в которых есть квартиры в продаже в этом районе в этом месяце	
* **rooms_offered_*_ratio** - доля квартир указанной в * комнатности от всех квартир в продаже в этом районе в этом месяце 	
* **median_price_sqm** - медианы цены за кв.м. в этом районе в этом месяце	
* **total_area_avg** - средний метраж продаваемой квартиры в этом районе в этом месяце	
* **studio_ratio** - доля объявлений о продаже студии среди всех объявлений в этом районе в этом месяце	
* **offer_count** - число объявлений о продаже квартир в этом районе в этом месяце	

In [18]:
df_price = pd.read_table(PATH + 'price_train.tsv')
df_price.month = df_price.month.apply(pd.to_datetime)
# df_price['next_month'] = df_price['month'].apply(lambda x: x + 
#                                                  relativedelta(months=1))
df_price.drop(columns=['offers_in_sites_ratio'], inplace=True)  # constss
print(df_price.month.min(), df_price.month.max())
get_zaic(df_price)
df_price.head(3)

2017-01-01 00:00:00 2019-08-01 00:00:00
2022-05-04 17:54:21
17.73 Mb
(166027, 14)


,apartment_ratio,unique_active_buildings_count,avg_price_sqm,month,unique_active_building_series_count,rooms_offered_3_ratio,city_quadkey,median_price_sqm,rooms_offered_4_more_ratio,rooms_offered_1_ratio,total_area_avg,studio_ratio,offer_count,rooms_offered_2_ratio
0,0.0,1,22727.273438,2018-02-01,1,0.0,120132230303223,22727.273438,0.0,0.0,44.0,0.0,1,1.0
1,0.0,1,22727.273438,2018-03-01,1,0.0,120132230303223,22727.273438,0.0,0.0,44.0,0.0,1,1.0
2,0.0,1,43478.261719,2018-06-01,1,0.0,120132230332230,43478.261719,0.0,1.0,34.5,0.0,1,0.0


Описание полей в датасете про базу домов price_housebase.tsv:

* **city_quadkey** - полигон, его координаты можно его узнать из скрипта quadkey.py
* **building_id** - id дома
* **unified_address** - адрес дома
* **building_series_id** - id серии дома
* **beauty_cnt** - количество салонов красоты в этом районе city_quadkey	
* **shopping_cnt** - количество магазинов в этом районе city_quadkey	 
* **cafe_restaurant_eating_out_cnt** - количество точек общественного питания в этом районе city_quadkey	
* **entertainment_cnt** - количество организаций в сфере развлечений в этом районе city_quadkey
* **sport_cnt** - количество спортивных организаций в этом районе city_quadkey
* **chain_cnt** - количество сетевых организаций в этом районе city_quadkey
* **groceries_and_everyday_items_cnt** - количество продуктовых магазинов в этом районе city_quadkey	
* **art_cnt** - количество организаций в сфере искусства (музеи, галереи и т.п.) в этом районе city_quadkey
* **healthcare_cnt** - количество организаций в сфере здравоохранения в этом районе city_quadkey	
* **flats_count** - количество квартир в доме	
* **laundry_and_repair_services_cnt** - количество прачечных и ремонтных мастерских в этом районе city_quadkey	
* **latitude, longitude** - координаты дома
* **expect_demolition** - дом входит в программу реновации и ожидает сноса
* **ceiling_height** - высота потолков в доме	
* **has_elevator** - наличие лифта в доме	
* **build_year** - год постройки дома

In [19]:
df_house = pd.read_table(PATH + 'price_housebase.tsv')
get_zaic(df_house)
df_house.expect_demolition = df_house.expect_demolition.apply(int)
df_house.has_elevator = df_house.has_elevator.apply(int)
df_house.head(3)

2022-05-04 17:54:21
45.33 Mb
(112203, 23)


,beauty_cnt,building_id,building_series_id,shopping_cnt,flats_count,latitude,building_type,cafe_restaurant_eating_out_cnt,unified_address,entertainment_cnt,sport_cnt,chain_cnt,groceries_and_everyday_items_cnt,art_cnt,site_id,healthcare_cnt,expect_demolition,longitude,ceiling_height,has_elevator,laundry_and_repair_services_cnt,build_year,city_quadkey
0,36.0,3031030817531317628,0,45.0,0,55.705219,BRICK,24.0,"Россия, Москва, 1-й Автозаводский проезд, 3",18.0,13.0,127.0,32.0,7.0,0,16.0,0,37.656994,0.0,1,38.0,1937,120310101302011
1,36.0,3031030817531317630,0,45.0,0,55.704845,BRICK,24.0,"Россия, Москва, 1-й Автозаводский проезд, 5",18.0,13.0,127.0,32.0,7.0,0,16.0,0,37.657295,0.0,0,38.0,1962,120310101302011
2,36.0,1764947998512909470,0,45.0,0,55.704697,UNKNOWN,24.0,"Россия, Москва, 1-й Автозаводский проезд, 7/1",18.0,13.0,127.0,32.0,7.0,0,16.0,0,37.658203,0.0,0,38.0,0,120310101302011


In [20]:
display(df_house.building_type.value_counts(normalize=True))
df_house.building_type = df_house.building_type.apply(lambda x: 'ANOTHER' if x in 
                                                      ['MONOLIT_BRICK', 
                                                       'WOOD', 
                                                       'METAL', 
                                                       'FERROCONCRETE'] else x)
df_house = pd.get_dummies(df_house, columns=['building_type'])
df_house.head(3)

BRICK            0.358359
PANEL            0.314751
UNKNOWN          0.200119
MONOLIT          0.081923
BLOCK            0.034206
MONOLIT_BRICK    0.008021
WOOD             0.002594
METAL            0.000018
FERROCONCRETE    0.000009
Name: building_type, dtype: float64

,beauty_cnt,building_id,building_series_id,shopping_cnt,flats_count,latitude,cafe_restaurant_eating_out_cnt,unified_address,entertainment_cnt,sport_cnt,chain_cnt,groceries_and_everyday_items_cnt,art_cnt,site_id,healthcare_cnt,expect_demolition,longitude,ceiling_height,has_elevator,laundry_and_repair_services_cnt,build_year,city_quadkey,building_type_ANOTHER,building_type_BLOCK,building_type_BRICK,building_type_MONOLIT,building_type_PANEL,building_type_UNKNOWN
0,36.0,3031030817531317628,0,45.0,0,55.705219,24.0,"Россия, Москва, 1-й Автозаводский проезд, 3",18.0,13.0,127.0,32.0,7.0,0,16.0,0,37.656994,0.0,1,38.0,1937,120310101302011,0,0,1,0,0,0
1,36.0,3031030817531317630,0,45.0,0,55.704845,24.0,"Россия, Москва, 1-й Автозаводский проезд, 5",18.0,13.0,127.0,32.0,7.0,0,16.0,0,37.657295,0.0,0,38.0,1962,120310101302011,0,0,1,0,0,0
2,36.0,1764947998512909470,0,45.0,0,55.704697,24.0,"Россия, Москва, 1-й Автозаводский проезд, 7/1",18.0,13.0,127.0,32.0,7.0,0,16.0,0,37.658203,0.0,0,38.0,0,120310101302011,0,0,0,0,0,1


In [21]:
useful_cols = [
    'beauty_cnt',
    'shopping_cnt',
    'flats_count',
    'cafe_restaurant_eating_out_cnt',
    'entertainment_cnt',
    'sport_cnt',
    'chain_cnt',
    'groceries_and_everyday_items_cnt',
    'art_cnt',
    'healthcare_cnt',
    'expect_demolition',
    'ceiling_height',
    'laundry_and_repair_services_cnt',
    'building_type_ANOTHER',
    'building_type_BLOCK',
    'building_type_BRICK',
    'building_type_MONOLIT',
    'building_type_PANEL',
    'building_type_UNKNOWN'
]
grouped_housebase = df_house[useful_cols + 
                             ['city_quadkey']].groupby("city_quadkey")

## Добавим информацию о домах

In [22]:
get_zaic(df_price)

df_price = df_price.merge(grouped_housebase.mean(), how="left", 
                          left_on="city_quadkey", 
                          suffixes=[None,"_bmean"], right_index=True)
df_price = df_price.merge(grouped_housebase.max(),  
                          how="left", left_on="city_quadkey", 
                          suffixes=[None,"_bmax"], right_index=True)
df_price = df_price.merge(grouped_housebase.min(),  
                          how="left", left_on="city_quadkey", 
                          suffixes=[None,"_bmin"], right_index=True)
print()
get_zaic(df_price)
df_price.head(3)

2022-05-04 17:54:22
17.73 Mb
(166027, 14)

2022-05-04 17:54:22
76.63 Mb
(166027, 71)


,apartment_ratio,unique_active_buildings_count,avg_price_sqm,month,unique_active_building_series_count,rooms_offered_3_ratio,city_quadkey,median_price_sqm,rooms_offered_4_more_ratio,rooms_offered_1_ratio,total_area_avg,studio_ratio,offer_count,rooms_offered_2_ratio,beauty_cnt,shopping_cnt,flats_count,cafe_restaurant_eating_out_cnt,entertainment_cnt,sport_cnt,chain_cnt,groceries_and_everyday_items_cnt,art_cnt,healthcare_cnt,expect_demolition,ceiling_height,laundry_and_repair_services_cnt,building_type_ANOTHER,building_type_BLOCK,building_type_BRICK,building_type_MONOLIT,building_type_PANEL,building_type_UNKNOWN,beauty_cnt_bmax,shopping_cnt_bmax,flats_count_bmax,cafe_restaurant_eating_out_cnt_bmax,entertainment_cnt_bmax,sport_cnt_bmax,chain_cnt_bmax,groceries_and_everyday_items_cnt_bmax,art_cnt_bmax,healthcare_cnt_bmax,expect_demolition_bmax,ceiling_height_bmax,laundry_and_repair_services_cnt_bmax,building_type_ANOTHER_bmax,building_type_BLOCK_bmax,building_type_BRICK_bmax,building_type_MONOLIT_bmax,building_type_PANEL_bmax,building_type_UNKNOWN_bmax,beauty_cnt_bmin,shopping_cnt_bmin,flats_count_bmin,cafe_restaurant_eating_out_cnt_bmin,entertainment_cnt_bmin,sport_cnt_bmin,chain_cnt_bmin,groceries_and_everyday_items_cnt_bmin,art_cnt_bmin,healthcare_cnt_bmin,expect_demolition_bmin,ceiling_height_bmin,laundry_and_repair_services_cnt_bmin,building_type_ANOTHER_bmin,building_type_BLOCK_bmin,building_type_BRICK_bmin,building_type_MONOLIT_bmin,building_type_PANEL_bmin,building_type_UNKNOWN_bmin
0,0.0,1,22727.273438,2018-02-01,1,0.0,120132230303223,22727.273438,0.0,0.0,44.0,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0,0.0,1.0,0,0,0,0,1,0,0.0,0.0,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0,0.0,1.0,0,0,0,0,1,0
1,0.0,1,22727.273438,2018-03-01,1,0.0,120132230303223,22727.273438,0.0,0.0,44.0,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0,0.0,1.0,0,0,0,0,1,0,0.0,0.0,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0,0.0,1.0,0,0,0,0,1,0
2,0.0,1,43478.261719,2018-06-01,1,0.0,120132230332230,43478.261719,0.0,1.0,34.5,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0,0.0,0.0,0,0,0,0,1,0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0,0.0,0.0,0,0,0,0,1,0


## Разбивка на train / test 

In [23]:
target = 'avg_price_sqm'
# target = 'avg_price_sqm_next'
model_cols = [x for x in df_price.columns if x not in [target, 'month']]

if SPLIT_TIPE == 'OOS':
    df_model = df_price[model_cols + [target]].copy()
else:
    df_model = df_price[model_cols + [target] + ['month']].copy()

get_zaic(df_model)
df_model.head(3)

2022-05-04 17:54:22
75.37 Mb
(166027, 70)


,apartment_ratio,unique_active_buildings_count,unique_active_building_series_count,rooms_offered_3_ratio,city_quadkey,median_price_sqm,rooms_offered_4_more_ratio,rooms_offered_1_ratio,total_area_avg,studio_ratio,offer_count,rooms_offered_2_ratio,beauty_cnt,shopping_cnt,flats_count,cafe_restaurant_eating_out_cnt,entertainment_cnt,sport_cnt,chain_cnt,groceries_and_everyday_items_cnt,art_cnt,healthcare_cnt,expect_demolition,ceiling_height,laundry_and_repair_services_cnt,building_type_ANOTHER,building_type_BLOCK,building_type_BRICK,building_type_MONOLIT,building_type_PANEL,building_type_UNKNOWN,beauty_cnt_bmax,shopping_cnt_bmax,flats_count_bmax,cafe_restaurant_eating_out_cnt_bmax,entertainment_cnt_bmax,sport_cnt_bmax,chain_cnt_bmax,groceries_and_everyday_items_cnt_bmax,art_cnt_bmax,healthcare_cnt_bmax,expect_demolition_bmax,ceiling_height_bmax,laundry_and_repair_services_cnt_bmax,building_type_ANOTHER_bmax,building_type_BLOCK_bmax,building_type_BRICK_bmax,building_type_MONOLIT_bmax,building_type_PANEL_bmax,building_type_UNKNOWN_bmax,beauty_cnt_bmin,shopping_cnt_bmin,flats_count_bmin,cafe_restaurant_eating_out_cnt_bmin,entertainment_cnt_bmin,sport_cnt_bmin,chain_cnt_bmin,groceries_and_everyday_items_cnt_bmin,art_cnt_bmin,healthcare_cnt_bmin,expect_demolition_bmin,ceiling_height_bmin,laundry_and_repair_services_cnt_bmin,building_type_ANOTHER_bmin,building_type_BLOCK_bmin,building_type_BRICK_bmin,building_type_MONOLIT_bmin,building_type_PANEL_bmin,building_type_UNKNOWN_bmin,avg_price_sqm
0,0.0,1,1,0.0,120132230303223,22727.273438,0.0,0.0,44.0,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0,0.0,1.0,0,0,0,0,1,0,0.0,0.0,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0,0.0,1.0,0,0,0,0,1,0,22727.273438
1,0.0,1,1,0.0,120132230303223,22727.273438,0.0,0.0,44.0,0.0,1,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0,0.0,1.0,0,0,0,0,1,0,0.0,0.0,0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0,0.0,1.0,0,0,0,0,1,0,22727.273438
2,0.0,1,1,0.0,120132230332230,43478.261719,0.0,1.0,34.5,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0,0.0,0.0,0,0,0,0,1,0,0.0,0.0,0,0.0,0.0,0.0,1.0,0.0,3.0,0.0,0,0.0,0.0,0,0,0,0,1,0,43478.261719


In [24]:
TEST_BORDER = 0.25
OOT_BORDER = pd.to_datetime('2019-01-01')
seed_val = 7575

if SPLIT_TIPE == 'OOS':
    print('OOS MODE')
    X_train, X_test, y_train, y_test = train_test_split(
        df_model[model_cols], df_model[target], 
        test_size=TEST_BORDER, random_state=seed_val)
else:
    print('OOT MODE')
    X_train = df_model[df_model.month < OOT_BORDER][model_cols]
    y_train = df_model[df_model.month < OOT_BORDER][target]
    X_test = df_model[df_model.month >= OOT_BORDER][model_cols]
    y_test = df_model[df_model.month >= OOT_BORDER][target]

OOS MODE
